<a href="https://colab.research.google.com/github/kavi139/Portfolios_project/blob/main/%D0%9B%D0%B8%D0%B7%D0%B0_%D0%90%D0%BB%D0%B5%D1%80%D1%82_%D1%82%D0%B5%D1%81%D1%82%D1%8B_10_%D0%B1%D0%B8%D0%B1%D0%BB%D0%B8%D0%BE%D1%82%D0%B5%D0%BA_%D0%B4%D0%BB%D1%8F_%D1%80%D0%B0%D1%81%D0%BF%D0%BE%D0%B7%D0%BD%D0%B0%D0%B2%D0%B0%D0%BD%D0%B8%D1%8F_%D0%BB%D0%B8%D1%86.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Загружаем данные

import gdown

gdown.download('https://drive.google.com/uc?id=1dYypMiRaT6zfyGq6TcE1Ket6p3x71HZl', None, quiet=False)

# Разархивация датасета
!unzip -qo "фото_поисковиков_подготовленные.zip" -d dataset/

Downloading...
From (original): https://drive.google.com/uc?id=1dYypMiRaT6zfyGq6TcE1Ket6p3x71HZl
From (redirected): https://drive.google.com/uc?id=1dYypMiRaT6zfyGq6TcE1Ket6p3x71HZl&confirm=t&uuid=80495f86-6d2e-418c-ab70-bdf93f2dbf99
To: /content/фото_поисковиков_подготовленные.zip
100%|██████████| 804M/804M [00:11<00:00, 68.4MB/s]


# Функция отображения размеченных фото

In [ ]:
# Отображение всех фото из указанной папки
from IPython.display import Image, display
import os

def display_images_from_folder(folder_path):
    # Получаем список файлов в папке
    files = os.listdir(folder_path)

    # Отображаем каждое изображение из списка файлов
    for file in files:
        # Полный путь к файлу
        file_path = os.path.join(folder_path, file)
        # Отображаем изображение
        display(Image(file_path))


In [ ]:
# Отображение одного файла
# from IPython.display import Image

# img_path = '/content/detected_faces_mtcnn/img_2211.jpg'
# Image(img_path)

# Модель распознования и разметки лиц на openCV

In [ ]:
# модель распознования лиц на openCV

!pip install opencv-python-headless

import os
import cv2
from google.colab.patches import cv2_imshow

# Пути к папкам с фотографиями
test_folders = [
    "/content/dataset/фото_поисковиков_подготовленные/фото_поисковиков_проверочные/01_ГБ_Проверочные",
    "/content/dataset/фото_поисковиков_подготовленные/фото_поисковиков_проверочные/02_ВК74_проверочные"
]

# Папка для сохранения обработанных фотографий
output_folder = "/content/detected_faces_cv2"

# Создаем папку для сохранения обработанных фотографий, если она не существует
os.makedirs(output_folder, exist_ok=True)

# Проходим по каждой папке с фотографиями
for folder in test_folders:
    # Получаем список файлов в папке
    files = os.listdir(folder)
    # Проходим по каждому файлу
    for file in files:
        # Полный путь к файлу
        file_path = os.path.join(folder, file)
        # Загружаем изображение
        img = cv2.imread(file_path)
        # Конвертируем изображение в оттенки серого
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        # Используем классификатор Хаара для обнаружения лиц на изображении
        face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
        faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
        # Проходим по каждому найденному лицу
        for (x, y, w, h) in faces:
            # Рисуем желтый квадрат вокруг лица
            cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 255), 2)
        # Сохраняем обработанное изображение
        output_file = os.path.join(output_folder, file)
        cv2.imwrite(output_file, img)

In [ ]:
# Отображаем размеченные фото для визуальной проверки качества разметки библиотеки
display_images_from_folder("/content/detected_faces_cv2")

##Тесты

In [ ]:
# Отображаем размеченные фото для визуальной проверки качества разметки библиотеки

def count_files_in_folder(folder_path):
    # Получаем список файлов в папке
    files = os.listdir(folder_path)
    # Подсчитываем общее количество файлов
    total_files = len(files)
    return total_files

# Путь к папке с обработанными изображениями
detected_faces_folder = "/content/detected_faces_cv2"

# Получаем общее количество файлов в папке
total_files = count_files_in_folder(detected_faces_folder)
print("Общее количество файлов в папке:", total_files)

# Получаем список файлов в папке
files = os.listdir(detected_faces_folder)

# Переменная для отслеживания ошибок
errors = []

# Проходим по каждому файлу
for file in files:
    # Полный путь к файлу
    file_path = os.path.join(detected_faces_folder, file)
    # Проверяем, является ли файл изображением
    if file.endswith(('.jpg', '.jpeg', '.png', '.gif')):
        # Загружаем изображение
        img = cv2.imread(file_path)
        # Проверяем размер изображения
        height, width, _ = img.shape
        if height > 3000 or width > 3000:
            # Уменьшаем размер изображения
            img = cv2.resize(img, (3000, int(3000 * height / width)))
        # Отображаем изображение
        cv2_imshow(img)
    else:
        # Добавляем файл в список ошибок
        errors.append(file)

In [ ]:
# Отображаем размеченные фотографии для проверки качества
import gc
# Путь к папке с обработанными изображениями
detected_faces_folder = "/content/detected_faces"

# Получаем список файлов в папке
files = os.listdir(detected_faces_folder)

# Проходим по каждому файлу
for file in files:
    gc.collect()
    # Полный путь к файлу
    file_path = os.path.join(detected_faces_folder, file)
    # Загружаем изображение
    img = cv2.imread(file_path)
    # Отображаем изображение
    cv2_imshow(img)


In [ ]:
def count_files_in_folder(folder_path):
    # Получаем список файлов в папке
    files = os.listdir(folder_path)
    # Подсчитываем общее количество файлов
    total_files = len(files)
    return total_files

# Путь к папке с обработанными изображениями
detected_faces_folder = "/content/detected_faces"

# Получаем общее количество файлов в папке
total_files = count_files_in_folder(detected_faces_folder)
print("Общее количество файлов в папке:", total_files)

# Получаем список файлов в папке
files = os.listdir(detected_faces_folder)

# Переменная для отслеживания ошибок
errors = []

# Проходим по каждому файлу
for file in files:
    # Полный путь к файлу
    file_path = os.path.join(detected_faces_folder, file)
    # Проверяем, является ли файл изображением
    if file.endswith(('.jpg', '.jpeg', '.png', '.gif')):
        # Загружаем изображение
        img = cv2.imread(file_path)
        # Отображаем изображение
        cv2_imshow(img)
    else:
        # Добавляем файл в список ошибок
        errors.append(file)

# Если есть файлы, не являющиеся изображениями, выводим сообщение об ошибке
if errors:
    print("Следующие файлы не являются изображениями и будут пропущены:")
    for error in errors:
        print(error)


In [ ]:
import matplotlib.pyplot as plt

# Путь к папке с обработанными изображениями
detected_faces_folder = "/content/detected_faces"

# Получаем список файлов в папке
files = os.listdir(detected_faces_folder)

# Переменная для отслеживания ошибок
errors = []

# Создаем горизонтальную прокрутку
plt.figure(figsize=(20, 10))

# Проходим по каждому файлу
for file in files:
    # Полный путь к файлу
    file_path = os.path.join(detected_faces_folder, file)
    # Проверяем, является ли файл изображением
    if file.endswith(('.jpg', '.jpeg', '.png', '.gif')):
        # Загружаем изображение
        img = cv2.imread(file_path)
        # Определяем размеры изображения для изменения размера
        height, width, _ = img.shape
        new_width = 2200  # Новая ширина изображения
        new_height = int(height * (new_width / width))  # Сохранение пропорций
        # Изменяем размер изображения
        resized_img = cv2.resize(img, (new_width, new_height))
        # Отображаем изображение
        plt.imshow(cv2.cvtColor(resized_img, cv2.COLOR_BGR2RGB))
        plt.axis('off')  # Отключаем оси координат
        plt.show()  # Показываем изображение
    else:
        # Добавляем файл в список ошибок
        errors.append(file)

# Если есть файлы, не являющиеся изображениями, выводим сообщение об ошибке
if errors:
    print("Следующие файлы не являются изображениями и будут пропущены:")
    for error in errors:
        print(error)



# Модель распознования и разметки лиц на pytorch

In [ ]:
!pip -q install facenet-pytorch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 55.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 62.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 85.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━

In [ ]:
import os
import cv2
import torch
from torchvision import transforms
from facenet_pytorch import MTCNN

# Пути к папкам с фотографиями
test_folders = [
    "/content/dataset/фото_поисковиков_подготовленные/фото_поисковиков_проверочные/01_ГБ_Проверочные",
    "/content/dataset/фото_поисковиков_подготовленные/фото_поисковиков_проверочные/02_ВК74_проверочные"
]

# Папка для сохранения обработанных фотографий
output_folder = "/content/detected_faces_pytorch"

# Создаем папку для сохранения обработанных фотографий, если она не существует
os.makedirs(output_folder, exist_ok=True)

# Загружаем модель MTCNN для обнаружения лиц
mtcnn = MTCNN()

# Проходим по каждой папке с фотографиями
for folder in test_folders:
    # Получаем список файлов в папке
    files = os.listdir(folder)
    # Проходим по каждому файлу
    for file in files:
        # Полный путь к файлу
        file_path = os.path.join(folder, file)
        # Загружаем изображение
        img = cv2.imread(file_path)
        # Конвертируем изображение в формат RGB (требуется для MTCNN)
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        # Применяем MTCNN для обнаружения лиц на изображении
        boxes, _ = mtcnn.detect(img_rgb)
        if boxes is not None:
            # Проходим по каждому обнаруженному лицу
            for box in boxes:
                x, y, w, h = map(int, box)
                # Уменьшаем размер прямоугольника
                scale = 0.8  # Примерный коэффициент уменьшения
                w_new = int(w * scale)
                h_new = int(h * scale)
                x_new = max(0, x + int((w - w_new) / 2))  # Ограничиваем координаты, чтобы не были отрицательными
                y_new = max(0, y + int((h - h_new) / 2))  # Ограничиваем координаты, чтобы не были отрицательными
                # Рисуем желтый квадрат вокруг лица
                cv2.rectangle(img, (x_new, y_new), (x_new + w_new, y_new + h_new), (0, 255, 255), 2)


        # Сохраняем обработанное изображение
        output_file = os.path.join(output_folder, file)
        cv2.imwrite(output_file, img)


In [ ]:
# Отображаем размеченные фото для визуальной проверки качества разметки библиотеки
display_images_from_folder("/content/detected_faces_pytorch")

## Тест функции отображения размеченных фото

In [ ]:
# Отображаем размеченные фото для визуальной проверки качества разметки библиотеки

def count_files_in_folder(folder_path):
    # Получаем список файлов в папке
    files = os.listdir(folder_path)
    # Подсчитываем общее количество файлов
    total_files = len(files)
    return total_files

# Путь к папке с обработанными изображениями
detected_faces_folder = "/content/detected_faces_pytorch"

# Получаем общее количество файлов в папке
total_files = count_files_in_folder(detected_faces_folder)
print("Общее количество файлов в папке:", total_files)

# Получаем список файлов в папке
files = os.listdir(detected_faces_folder)

# Переменная для отслеживания ошибок
errors = []

# Проходим по каждому файлу
for file in files:
    # Полный путь к файлу
    file_path = os.path.join(detected_faces_folder, file)
    # Проверяем, является ли файл изображением
    if file.endswith(('.jpg', '.jpeg', '.png', '.gif')):
        # Загружаем изображение
        img = cv2.imread(file_path)
        # Проверяем размер изображения
        height, width, _ = img.shape
        if height > 3000 or width > 3000:
            # Уменьшаем размер изображения
            img = cv2.resize(img, (3000, int(3000 * height / width)))
        # Отображаем изображение
        cv2_imshow(img)
    else:
        # Добавляем файл в список ошибок
        errors.append(file)

# Модель распознования и разметки лиц на dlib

In [ ]:
# Устанавливаем необходимые библиотеки
!pip -q install dlib

import os
import cv2
import dlib

# Пути к папкам с фотографиями
test_folders = [
    "/content/dataset/фото_поисковиков_подготовленные/фото_поисковиков_проверочные/01_ГБ_Проверочные",
    "/content/dataset/фото_поисковиков_подготовленные/фото_поисковиков_проверочные/02_ВК74_проверочные"
]

# Папка для сохранения обработанных фотографий
output_folder = "/content/detected_faces_dlib"

# Создаем папку для сохранения обработанных фотографий, если она не существует
os.makedirs(output_folder, exist_ok=True)

# Загружаем модель для обнаружения лиц
detector = dlib.get_frontal_face_detector()

# Проходим по каждой папке с фотографиями
for folder in test_folders:
    # Получаем список файлов в папке
    files = os.listdir(folder)
    # Проходим по каждому файлу
    for file in files:
        # Полный путь к файлу
        file_path = os.path.join(folder, file)
        # Загружаем изображение
        img = cv2.imread(file_path)
        # Преобразуем изображение в оттенки серого (требуется для dlib)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        # Используем детектор лиц для обнаружения лиц на изображении
        faces = detector(gray)
        # Проходим по каждому найденному лицу
        for face in faces:
            # Получаем координаты лица
            x, y, w, h = face.left(), face.top(), face.width(), face.height()
            # Рисуем желтый квадрат вокруг лица
            cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 255), 2)
        # Сохраняем обработанное изображение
        output_file = os.path.join(output_folder, file)
        cv2.imwrite(output_file, img)


In [ ]:
# Отображаем размеченные фото для визуальной проверки качества разметки библиотеки
display_images_from_folder("/content/detected_faces_dlib")

# Модель распознования и разметки лиц на DeepFace

In [ ]:
# Установка необходимых библиотек
!pip -q install deepface

import os
import cv2
from deepface import DeepFace

In [ ]:
# Пути к папкам с фотографиями
test_folders = [
    "/content/dataset/фото_поисковиков_подготовленные/фото_поисковиков_проверочные/01_ГБ_Проверочные",
    "/content/dataset/фото_поисковиков_подготовленные/фото_поисковиков_проверочные/02_ВК74_проверочные"
]

# Папка для сохранения обработанных фотографий
output_folder = "/content/detected_faces_deepface"

# Создаем папку для сохранения обработанных фотографий, если она не существует
os.makedirs(output_folder, exist_ok=True)

# Проходим по каждой папке с фотографиями
for folder in test_folders:
    # Получаем список файлов в папке
    files = os.listdir(folder)
    # Проходим по каждому файлу
    for file in files:
        # Полный путь к файлу
        file_path = os.path.join(folder, file)
        # Загружаем изображение
        img = cv2.imread(file_path)
        # Обнаруживаем лица на изображении с помощью DeepFace
        detected_faces = DeepFace.extract_faces(img, detector_backend='opencv')
        # Рисуем прямоугольники вокруг обнаруженных лиц
        for face in detected_faces:
            box = face['facial_area']
            x, y, w, h = box['x'], box['y'], box['w'], box['h']
            cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 255), 2)
        # Сохраняем обработанное изображение
        output_file = os.path.join(output_folder, file)
        cv2.imwrite(output_file, img)

In [ ]:
# Отображаем размеченные фото для визуальной проверки качества разметки библиотеки
display_images_from_folder("/content/detected_faces_deepface")

# Модель распознования и разметки лиц на MobileNetV2, MTCNN

In [ ]:
# Устанавливаем необходимые библиотеки
!pip install opencv-python-headless
!pip install mtcnn

import os
import cv2
from mtcnn import MTCNN

# Пути к папкам с фотографиями
test_folders = [
    "/content/dataset/фото_поисковиков_подготовленные/фото_поисковиков_проверочные/01_ГБ_Проверочные",
    "/content/dataset/фото_поисковиков_подготовленные/фото_поисковиков_проверочные/02_ВК74_проверочные"
]

# Папка для сохранения обработанных фотографий
output_folder = "/content/detected_faces_mtcnn"

# Создаем папку для сохранения обработанных фотографий, если она не существует
os.makedirs(output_folder, exist_ok=True)

# Создаем экземпляр детектора MTCNN
detector = MTCNN()

# Проходим по каждой папке с фотографиями
for folder in test_folders:
    # Получаем список файлов в папке
    files = os.listdir(folder)
    # Проходим по каждому файлу
    for file in files:
        # Полный путь к файлу
        file_path = os.path.join(folder, file)
        # Загружаем изображение
        img = cv2.imread(file_path)
        # Преобразуем изображение в формат RGB (требуется для MTCNN)
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        # Используем детектор лиц для обнаружения лиц на изображении
        faces = detector.detect_faces(img_rgb)
        # Проходим по каждому найденному лицу
        for face in faces:
            # Получаем координаты лица
            x, y, w, h = face['box']
            # Рисуем желтый квадрат вокруг лица
            cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 255), 2)
        # Сохраняем обработанное изображение
        output_file = os.path.join(output_folder, file)
        cv2.imwrite(output_file, img)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 14.4 MB/s eta 0:00:00
2/2 [==============================] - 0s 179ms/step


In [ ]:
# Отображаем размеченные фото для визуальной проверки качества разметки библиотеки
display_images_from_folder("/content/detected_faces_mtcnn")

In [ ]:
!pip install -q opencv-python-headless
!pip install -q mtcnn

import os
import cv2
from mtcnn import MTCNN

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 18.6 MB/s eta 0:00:00


In [ ]:
# Пути к папкам с фотографиями
test_folders = [
    '/content/dataset/фото_поисковиков_подготовленные/фото_поисковиков_проверочные/01_ГБ_Проверочные',
    '/content/dataset/фото_поисковиков_подготовленные/фото_поисковиков_проверочные/02_ВК74_проверочные',
    '/content/dataset/фото_поисковиков_подготовленные/фото_поисковиков_проверочные/03_Клёцин_проверочные',
    '/content/dataset/фото_поисковиков_подготовленные/фото_поисковиков_проверочные/04_Незабудка_проверочные',
    '/content/dataset/фото_поисковиков_подготовленные/фото_поисковиков_проверочные/05_Нибилунг_проверочные',
    '/content/dataset/фото_поисковиков_подготовленные/фото_поисковиков_проверочные/06_Василий_проверочные',
    '/content/dataset/фото_поисковиков_подготовленные/фото_поисковиков_проверочные/07_Физрук_проверочные',
    '/content/dataset/фото_поисковиков_подготовленные/фото_поисковиков_проверочные/08_Ватсон_проверочные',
    '/content/dataset/фото_поисковиков_подготовленные/фото_поисковиков_проверочные/09_Кеслер_проверочные',
    '/content/dataset/фото_поисковиков_подготовленные/фото_поисковиков_проверочные/10_Ёж_проверочные',
    '/content/dataset/фото_поисковиков_подготовленные/фото_поисковиков_проверочные/11_Якимчинк_проверочные',
    '/content/dataset/фото_поисковиков_подготовленные/фото_поисковиков_проверочные/12_Чайка_проверочные',
    '/content/dataset/фото_поисковиков_подготовленные/фото_поисковиков_проверочные/13_Весна_проверочные',
    '/content/dataset/фото_поисковиков_подготовленные/фото_поисковиков_проверочные/14_МиСи_проверочные',
    '/content/dataset/фото_поисковиков_подготовленные/фото_поисковиков_проверочные/15_Крапива_проверочные',
    '/content/dataset/фото_поисковиков_подготовленные/фото_поисковиков_проверочные/16_Мохнатый_проверочные',
    '/content/dataset/фото_поисковиков_подготовленные/фото_поисковиков_проверочные/17_Пифа_проверочные',
    '/content/dataset/фото_поисковиков_подготовленные/фото_поисковиков_проверочные/18_Семён_проверочные',
    '/content/dataset/фото_поисковиков_подготовленные/фото_поисковиков_проверочные/19_Будагов_проверочные',
    '/content/dataset/фото_поисковиков_подготовленные/фото_поисковиков_проверочные/20_Дулин_проверочные',
    '/content/dataset/фото_поисковиков_подготовленные/фото_поисковиков_проверочные/21_Лентяй_проверочные',
    '/content/dataset/фото_поисковиков_подготовленные/фото_поисковиков_проверочные/22_Снежный_Барс_проверочные',
    '/content/dataset/фото_поисковиков_подготовленные/фото_поисковиков_проверочные/23_Евген_проверочные',
    '/content/dataset/фото_поисковиков_подготовленные/фото_поисковиков_проверочные/24_Воркута_проверочные',
   '/content/dataset/фото_поисковиков_подготовленные/фото_поисковиков_проверочные/25_Веснушка_проверочные'
]

# Папка для сохранения обработанных фотографий
output_folder = "/content/detected_faces_mtcnn_ful"

# Создаем папку для сохранения обработанных фотографий, если она не существует
os.makedirs(output_folder, exist_ok=True)

# Создаем экземпляр детектора MTCNN
detector = MTCNN()

# Проходим по каждой папке с фотографиями
for folder in test_folders:
    # Получаем список файлов в папке
    files = os.listdir(folder)
    # Проходим по каждому файлу
    for file in files:
        # Полный путь к файлу
        file_path = os.path.join(folder, file)
        # Загружаем изображение
        img = cv2.imread(file_path)
        # Преобразуем изображение в формат RGB (требуется для MTCNN)
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        # Используем детектор лиц для обнаружения лиц на изображении
        faces = detector.detect_faces(img_rgb)
        # Проходим по каждому найденному лицу
        for face in faces:
            # Получаем координаты лица
            x, y, w, h = face['box']
            # Рисуем желтый квадрат вокруг лица
            cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 255), 2)
        # Сохраняем обработанное изображение
        output_file = os.path.join(output_folder, file)
        cv2.imwrite(output_file, img)

4/4 [==============================] - 0s 4ms/step


In [ ]:
# Отображаем размеченные фото для визуальной проверки качества разметки библиотеки
display_images_from_folder("/content/detected_faces_mtcnn_ful")

In [ ]:
# Скачиваем размеченые фото
from google.colab import files

# Путь к папке с фотографиями
folder_path = '/content/detected_faces_mtcnn_ful'

# Получаем список файлов в папке
files_list = os.listdir(folder_path)

# Скачиваем файлы один за другим
for file_name in files_list:
    file_path = os.path.join(folder_path, file_name)
    files.download(file_path)


In [ ]:
import zipfile
import os

# Путь к папке с фотографиями
folder_path = '/content/detected_faces_mtcnn_ful'

# Путь, по которому будет сохранен архив
zip_file_path = '/content/detected_faces_mtcnn_ful.zip'

# Создаем архив
with zipfile.ZipFile(zip_file_path, 'w') as zipf:
    for root, _, files in os.walk(folder_path):
        for file in files:
            zipf.write(os.path.join(root, file), os.path.relpath(os.path.join(root, file), folder_path))

# Скачиваем архив
from google.colab import files
files.download(zip_file_path)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Модель распознования и разметки лиц на YoloV5, V8, V9

In [ ]:
#YOLOv5
# Устанавливаем необходимые библиотеки
!pip install -U -q yolov5

import os
import torch
import cv2
from tqdm.notebook import tqdm

# Пути к папкам с фотографиями
test_folders = [
    "/content/dataset/фото_поисковиков_подготовленные/фото_поисковиков_проверочные/01_ГБ_Проверочные",
    "/content/dataset/фото_поисковиков_подготовленные/фото_поисковиков_проверочные/02_ВК74_проверочные"
]

# Папка для сохранения обработанных фотографий
output_folder = "/content/detected_faces_yolov5"

# Создаем папку для сохранения обработанных фотографий, если она не существует
os.makedirs(output_folder, exist_ok=True)

# Загружаем предварительно обученную модель YOLOv5 для обнаружения лиц
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)

# Проходим по каждой папке с фотографиями
for folder in test_folders:
    # Получаем список файлов в папке
    files = os.listdir(folder)
    # Проходим по каждому файлу
    for file in tqdm(files, desc=folder):
        # Полный путь к файлу
        file_path = os.path.join(folder, file)
        # Загружаем изображение
        img = cv2.imread(file_path)
        # Переводим изображение в формат RGB
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        # Обнаруживаем лица на изображении с помощью модели YOLOv5
        results = model(img_rgb)
        # Рисуем прямоугольники вокруг обнаруженных лиц
        for detection in results.xyxy[0]:
            x1, y1, x2, y2, conf, cls = detection.tolist()
            if conf > 0.5:  # Устанавливаем порог уверенности
                cv2.rectangle(img, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 255), 2)
        # Сохраняем обработанное изображение
        output_file = os.path.join(output_folder, file)
        cv2.imwrite(output_file, img)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.4/953.4 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 749.1/749.1 kB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.4/88.4 kB 11.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 43.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.2/82.2 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.3/158.3 kB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.7/178.7 

/usr/local/lib/python3.10/dist-packages/torch/hub.py:294: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip
YOLOv5 🚀 2024-4-5 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)

100%|██████████| 14.1M/14.1M [00:00<00:00, 194MB/s]

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


/content/dataset/фото_поисковиков_подготовленные/фото_поисковиков_проверочные/01_ГБ_Проверочные:   0%|        …

WARNING ⚠️ NMS time limit 0.550s exceeded


/content/dataset/фото_поисковиков_подготовленные/фото_поисковиков_проверочные/02_ВК74_проверочные:   0%|      …

In [ ]:
# Отображаем размеченные фото для визуальной проверки качества разметки библиотеки
display_images_from_folder("/content/detected_faces_yolov5")

##YOLOv8

In [ ]:
!pip install git+https://github.com/ultralytics/yolov8s.git
# https://github.com/ultralytics/assets/releases/download/v8.1.0/yolov8s.pt

In [ ]:
!pip install -U -q yolov8

In [ ]:
from ultralytics import YOLO

In [ ]:
!pip install -q torch torchvision opencv-python

In [ ]:
!pip install -e /path/to/

ERROR: /path/to/ is not a valid editable requirement. It should either be a path to a local project or a VCS URL (beginning with bzr+http, bzr+https, bzr+ssh, bzr+sftp, bzr+ftp, bzr+lp, bzr+file, git+http, git+https, git+ssh, git+git, git+file, hg+file, hg+http, hg+https, hg+ssh, hg+static-http, svn+ssh, svn+http, svn+https, svn+svn, svn+file).


In [ ]:
import os
import cv2
import torch
from tqdm.notebook import tqdm

# Define the path to the YOLOv8 repository
yolov8_repo_path = "/content/yolov8"

# Clone the YOLOv8 repository
if not os.path.exists(yolov8_repo_path):
    os.system(f"git clone https://github.com/ultralytics/yolov8.git {yolov8_repo_path}")

# Load the YOLOv8 model
model = torch.hub.load('ultralytics/yolov8', 'yolov8s', pretrained=True, device='cuda')

# Пути к папкам с фотографиями
test_folders = [
    "/content/dataset/фото_поисковиков_подготовленные/фото_поисковиков_проверочные/01_ГБ_Проверочные",
    "/content/dataset/фото_поисковиков_подготовленные/фото_поисковиков_проверочные/02_ВК74_проверочные"
]

# Папка для сохранения обработанных фотографий
output_folder = "/content/detected_faces_yolov8"

# Создаем папку для сохранения обработанных фотографий, если она не существует
os.makedirs(output_folder, exist_ok=True)

# Переходим на GPU, если доступна
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

# Проходим по каждой папке с фотографиями
for folder in test_folders:
    # Получаем список файлов в папке
    files = os.listdir(folder)
    # Проходим по каждому файлу
    for file in tqdm(files, desc=folder):
        # Полный путь к файлу
        file_path = os.path.join(folder, file)
        # Загружаем изображение
        img = cv2.imread(file_path)
        # Переводим изображение в формат RGB
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        # Обнаруживаем лица на изображении с помощью модели YOLOv8
        results = model(img_rgb)
        # Рисуем прямоугольники вокруг обнаруженных лиц
        for detection in results.xyxy[0]:
            x1, y1, x2, y2, conf, cls = detection.tolist()
            if conf > 0.5:  # Устанавливаем порог уверенности
                cv2.rectangle(img, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 255), 2)

HTTPError: HTTP Error 404: Not Found

##YOLOv9

In [ ]:
# YOLOv9
from ultralytics import YOLO


# Модель распознования и разметки лиц на ResNet

In [ ]:
import os
import cv2
import torch
import torchvision.transforms as transforms
from PIL import Image

# Загружаем предварительно обученную модель ResNet
model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=True)
model.eval()

# Определяем преобразования для входных изображений модели ResNet
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Размер входного изображения ResNet
    transforms.ToTensor(),  # Преобразование в тензор
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Нормализация
])

# Пути к папкам с фотографиями
test_folders = [
    "/content/dataset/фото_поисковиков_подготовленные/фото_поисковиков_проверочные/01_ГБ_Проверочные",
    "/content/dataset/фото_поисковиков_подготовленные/фото_поисковиков_проверочные/02_ВК74_проверочные"
]

# Папка для сохранения обработанных фотографий
output_folder = "/content/detected_faces_resnet"

# Создаем папку для сохранения обработанных фотографий, если она не существует
os.makedirs(output_folder, exist_ok=True)

# Проходим по каждой папке с фотографиями
for folder in test_folders:
    # Получаем список файлов в папке
    files = os.listdir(folder)
    # Проходим по каждому файлу
    for file in files:
        # Полный путь к файлу
        file_path = os.path.join(folder, file)
        # Загружаем изображение
        img = Image.open(file_path)
        # Применяем преобразования к изображению
        img_tensor = transform(img).unsqueeze(0)  # Добавляем размерность батча
        # Получаем предсказания модели ResNet
        with torch.no_grad():
            outputs = model(img_tensor)
        # Используем softmax для получения вероятностей классов
        probabilities = torch.nn.functional.softmax(outputs[0], dim=0)
        # Если вероятность лица выше порогового значения, считаем это лицом
        if probabilities[0] > 0.3:
            # Рисуем желтый квадрат на изображении (заглушка для примера)
            img_cv2 = cv2.imread(file_path)
            h, w, _ = img_cv2.shape
            cv2.rectangle(img_cv2, (0, 0), (w, h), (0, 255, 255), 2)
            # Сохраняем обработанное изображение
            output_file = os.path.join(output_folder, file)
            cv2.imwrite(output_file, img_cv2)


Downloading: "https://github.com/pytorch/vision/zipball/v0.10.0" to /root/.cache/torch/hub/v0.10.0.zip
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 154MB/s]


In [ ]:
# Отображаем размеченные фото для визуальной проверки качества разметки библиотеки
display_images_from_folder("/content/detected_faces_resnet")

# Модель распознования и разметки лиц на EfficientNet


In [ ]:
# Установим библиотеку timm
!pip install timm

import os
import cv2
import torch
import torchvision.transforms as transforms
import timm
from PIL import Image

# Загружаем предварительно обученную модель EfficientNet
model = timm.create_model('efficientnet_b0', pretrained=True)
model.eval()

# Определяем преобразования для входных изображений модели EfficientNet
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Размер входного изображения EfficientNet
    transforms.ToTensor(),  # Преобразование в тензор
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Нормализация
])

# Пути к папкам с фотографиями
test_folders = [
    "/content/dataset/фото_поисковиков_подготовленные/фото_поисковиков_проверочные/01_ГБ_Проверочные",
    "/content/dataset/фото_поисковиков_подготовленные/фото_поисковиков_проверочные/02_ВК74_проверочные"
]

# Папка для сохранения обработанных фотографий
output_folder = "/content/detected_faces_efficientnet"

# Создаем папку для сохранения обработанных фотографий, если она не существует
os.makedirs(output_folder, exist_ok=True)

# Проходим по каждой папке с фотографиями
for folder in test_folders:
    # Получаем список файлов в папке
    files = os.listdir(folder)
    # Проходим по каждому файлу
    for file in files:
        # Полный путь к файлу
        file_path = os.path.join(folder, file)
        # Загружаем изображение
        img = Image.open(file_path)
        # Применяем преобразования к изображению
        img_tensor = transform(img).unsqueeze(0)  # Добавляем размерность батча
        # Получаем предсказания модели EfficientNet
        with torch.no_grad():
            outputs = model(img_tensor)
        # Используем softmax для получения вероятностей классов
        probabilities = torch.nn.functional.softmax(outputs, dim=1)
        # Если вероятность лица выше порогового значения, считаем это лицом
        if probabilities[0, 0] > 0.3:
            # Рисуем желтый квадрат на изображении (заглушка для примера)
            img_cv2 = cv2.imread(file_path)
            h, w, _ = img_cv2.shape
            cv2.rectangle(img_cv2, (0, 0), (w, h), (0, 255, 255), 2)
            # Сохраняем обработанное изображение
            output_file = os.path.join(output_folder, file)
            cv2.imwrite(output_file, img_cv2)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 15.8 MB/s eta 0:00:00


model.safetensors:   0%|          | 0.00/21.4M [00:00<?, ?B/s]

In [ ]:
# Отображаем размеченные фото для визуальной проверки качества разметки библиотеки
display_images_from_folder("/content/detected_faces_efficientnet")

# Модель распознования и разметки лиц на InsightFace

In [ ]:
# Установка библиотеки InsightFace
!pip -q install insightface
!pip -q install onnxruntime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.5/439.5 kB 3.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 49.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 11.4 MB/s eta 0:00:00


In [ ]:
import subprocess

def update_libraries():
    try:
        # Обновляем библиотеки с помощью pip
        subprocess.check_call(['pip', 'install', '--upgrade', 'insightface', 'onnxruntime'])
        print("Библиотеки успешно обновлены.")
    except Exception as e:
        print("Ошибка при обновлении библиотек:", e)

# Вызываем функцию для обновления библиотек
update_libraries()


Библиотеки успешно обновлены.


In [ ]:
import os
import cv2
from insightface.app import FaceAnalysis

# Создание экземпляра модели для обнаружения лиц
model = FaceAnalysis()

# Пути к папкам с фотографиями
test_folders = [
    "/content/dataset/фото_поисковиков_подготовленные/фото_поисковиков_проверочные/01_ГБ_Проверочные",
    "/content/dataset/фото_поисковиков_подготовленные/фото_поисковиков_проверочные/02_ВК74_проверочные"
]

# Папка для сохранения обработанных фотографий
output_folder = "/content/detected_faces_insightface"

# Создаем папку для сохранения обработанных фотографий, если она не существует
os.makedirs(output_folder, exist_ok=True)

# Проходим по каждой папке с фотографиями
for folder in test_folders:
    # Получаем список файлов в папке
    files = os.listdir(folder)
    # Проходим по каждому файлу
    for file in files:
        # Полный путь к файлу
        file_path = os.path.join(folder, file)
        # Загружаем изображение
        img = cv2.imread(file_path)
        # Применяем модель для обнаружения лиц
        faces = model.get(img)
        # Проходим по каждому обнаруженному лицу
        for face in faces:
            # Получаем координаты лица
            x1, y1, x2, y2 = face.bbox.astype(int)
            # Рисуем прямоугольник вокруг лица
            cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 255), 2)
        # Сохраняем обработанное изображение
        output_file = os.path.join(output_folder, file)
        cv2.imwrite(output_file, img)


download_path: /root/.insightface/models/buffalo_l


100%|██████████| 281857/281857 [00:05<00:00, 50044.92KB/s]
/usr/local/lib/python3.10/dist-packages/onnxruntime/capi/onnxruntime_inference_collection.py:69: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'AzureExecutionProvider, CPUExecutionProvider'
  warnings.warn(


Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/w600k_r50.onnx recognition ['None', 3, 112, 112] 127.5 127.5


AssertionError: 

In [ ]:
import os
import cv2
from insightface.app import FaceAnalysis

# Создание экземпляра модели для обнаружения лиц
model = FaceAnalysis()

# Установка размера входного изображения для модели retinaface
model.retinaface.set_input_size(640)

# Пути к папкам с фотографиями
test_folders = [
    "/content/dataset/фото_поисковиков_подготовленные/фото_поисковиков_проверочные/01_ГБ_Проверочные",
    "/content/dataset/фото_поисковиков_подготовленные/фото_поисковиков_проверочные/02_ВК74_проверочные"
]

# Папка для сохранения обработанных фотографий
output_folder = "/content/detected_faces_insightface"

# Создаем папку для сохранения обработанных фотографий, если она не существует
os.makedirs(output_folder, exist_ok=True)

# Проходим по каждой папке с фотографиями
for folder in test_folders:
    # Получаем список файлов в папке
    files = os.listdir(folder)
    # Проходим по каждому файлу
    for file in files:
        # Полный путь к файлу
        file_path = os.path.join(folder, file)
        # Загружаем изображение
        img = cv2.imread(file_path)
        # Применяем модель для обнаружения лиц
        faces = model.get(img, max_num=10)
        # Проходим по каждому обнаруженному лицу
        for face in faces:
            # Получаем координаты лица
            x1, y1, x2, y2 = face.bbox.astype(int)
            # Рисуем прямоугольник вокруг лица
            cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 255), 2)
        # Сохраняем обработанное изображение
        output_file = os.path.join(output_folder, file)
        cv2.imwrite(output_file, img)


Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/w600k_r50.onnx recognition ['None', 3, 112, 112] 127.5 127.5


AttributeError: 'FaceAnalysis' object has no attribute 'retinaface'